<a href="https://colab.research.google.com/github/totminaekaterina/RUSSE-2022-Detoxification/blob/main/ruprompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install ruprompts 

     |████████████████████████████████| 3.5 MB 8.5 MB/s 
     |████████████████████████████████| 895 kB 36.6 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 596 kB 47.6 MB/s 
     |████████████████████████████████| 6.8 MB 29.4 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: typeguard
    Found existing installation: typeguard 2.7.1
    Uninstalling typeguard-2.7.1:
      Successfully uninstalled typeguard-2.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arviz 0.11.4 require

In [2]:
!wget https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/dev.tsv
!wget https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/train.tsv

--2022-02-11 11:47:16--  https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200691 (196K) [text/plain]
Saving to: ‘dev.tsv’

dev.tsv             100%[===================>] 195.99K  --.-KB/s    in 0.02s   

2022-02-11 11:47:16 (8.24 MB/s) - ‘dev.tsv’ saved [200691/200691]

--2022-02-11 11:47:16--  https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1902888 (1.8M) 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

df = pd.read_csv("train.tsv", sep="\t")
df.drop(["index"], axis=1, inplace=True)
df.to_csv("train.tsv", index=False, sep="\t")

## Training

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Load the dataset:

In [6]:
!pip install datasets

     |████████████████████████████████| 311 kB 4.7 MB/s 
     |████████████████████████████████| 133 kB 36.8 MB/s 
     |████████████████████████████████| 1.1 MB 11.9 MB/s 
     |████████████████████████████████| 243 kB 33.7 MB/s 
     |████████████████████████████████| 94 kB 1.9 MB/s 
     |████████████████████████████████| 144 kB 31.3 MB/s 
     |████████████████████████████████| 271 kB 25.3 MB/s 


In [7]:
from datasets import load_dataset

datasets = load_dataset("csv", data_files={"train": "train.tsv", "validation": "dev.tsv"}, sep="\t")
train_dataset = datasets["train"]
valid_dataset = datasets["validation"]

Using custom data configuration default-29b59a8518e45d2f


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-29b59a8518e45d2f/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Load the backbone:

In [9]:
from transformers import GPT2LMHeadModel, AutoTokenizer

backbone_id = "/content/drive/MyDrive/model/checkpoint-2405"

model = GPT2LMHeadModel.from_pretrained(backbone_id)
tokenizer = AutoTokenizer.from_pretrained(backbone_id, pad_token="<pad>", eos_token="<pad>")

Define the prompt format:

In [10]:
from ruprompts import PromptFormat

prompt_format = PromptFormat("<P*100>{toxic_comment}<P*20>")

Define the parametrization of trainable embeddings:

In [11]:
from ruprompts import TensorPromptProvider
from transformers import set_seed

set_seed(1)

prompt_provider = TensorPromptProvider()

In [12]:
from ruprompts import Prompt

prompt = Prompt(prompt_format, prompt_provider)
prompt.patch(model, tokenizer)

In [13]:
from ruprompts import Text2TextPreprocessor

preprocessor = Text2TextPreprocessor(
    prompt_format=prompt_format,
    tokenizer=tokenizer,
    target_field="neutral_comment1",
    max_tokens=1792,
    truncation_field="toxic_comment",
)

train_dataset = train_dataset.map(preprocessor)
valid_dataset = valid_dataset.map(preprocessor)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

Define training arguments:

In [21]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=".",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,
    eval_steps=3000,
    save_steps=3000,
    logging_steps=3000,
    evaluation_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",
    save_total_limit=2,
    metric_for_best_model="eval_loss",
    learning_rate=0.1,
    max_steps=30000,
    report_to="tensorboard",
    logging_dir="logs",
    seed=1,
)

PyTorch: setting up devices


Choose optimization options:

In [22]:
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(prompt_provider.parameters(), lr=training_args.learning_rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=200,
    num_training_steps=training_args.max_steps,
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Define the callbacks and start training:

In [23]:
from transformers import Trainer
from ruprompts.callbacks import (
    FreezeTransformerUnfreezePrompt,
    ReduceCheckpoint,
    SavePretrainedPrompt,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=preprocessor.collate_fn(),
    optimizers=(optimizer, scheduler),
    callbacks=[FreezeTransformerUnfreezePrompt(), ReduceCheckpoint(), SavePretrainedPrompt(prompt)],
)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs
The following columns in the training set  don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: neutral_comment1, neutral_comment2, neutral_comment3, toxic_comment.
***** Running training *****
  Num examples = 6948
  Num Epochs = 9
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss,Validation Loss
3000,1.383000,1.758443
6000,1.098700,1.601526
9000,1.080700,1.590715
12000,1.042600,1.580088
15000,0.984400,1.558293
18000,0.986400,1.514436
21000,0.944700,1.500569
24000,0.939700,1.484763
27000,0.927600,1.506148
30000,0.929000,1.502401


The following columns in the evaluation set  don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: neutral_comment1, neutral_comment2, neutral_comment3, toxic_comment.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 2
Saving model checkpoint to ./checkpoint-3000
Configuration saved in ./checkpoint-3000/config.json
Model weights saved in ./checkpoint-3000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: neutral_comment1, neutral_comment2, neutral_comment3, toxic_comment.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 2
Saving model checkpoint to ./checkpoint-6000
Configuration saved in ./checkpoint-6000/config.json
Model weights saved in ./checkpoint-6000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: neutral_com

TrainOutput(global_step=30000, training_loss=1.0316907633463541, metrics={'train_runtime': 13266.8333, 'train_samples_per_second': 4.523, 'train_steps_per_second': 2.261, 'total_flos': 1.7832345555935232e+16, 'train_loss': 1.0316907633463541, 'epoch': 8.64})

## Inference

Load prompt from the last checkpoint:

In [25]:
from transformers import pipeline

prompt = Prompt.from_pretrained("/content/drive/MyDrive/model/checkpoint-30000")

ppln = pipeline("text2text-generation-with-prompt", prompt=prompt, model=model, tokenizer=tokenizer, device=0)

In [26]:
ppln({"toxic_comment": "Ублюдок, мать твою, а ну иди сюда"}, do_sample=False)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'А ну иди сюда'}]

Run inference:

In [27]:
!wget https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/test.tsv

--2022-02-11 15:35:29--  https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104462 (102K) [text/plain]
Saving to: ‘test.tsv’

test.tsv            100%[===================>] 102.01K  --.-KB/s    in 0.02s   

2022-02-11 15:35:29 (5.00 MB/s) - ‘test.tsv’ saved [104462/104462]



In [28]:
test = pd.read_csv("test.tsv", sep="\t")
test.to_csv("test.tsv", index=False, sep="\t")

In [29]:
from tqdm import tqdm
import transformers

transformers.logging.set_verbosity_error()

beam_count = 5

predictions = []
    
for i in tqdm(valid_dataset["toxic_comment"]):
    options = ppln(
        {"toxic_comment": i},
        do_sample=False,
        num_beams=beam_count,
        num_return_sequences=beam_count,
    )

    options = [i["generated_text"].replace("<pad>", "") for i in options]
    answer = sorted(options, key=len)[-1]  # get longest answer
    predictions.append(answer)

  1%|          | 9/800 [00:07<09:49,  1.34it/s]/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:978: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
100%|██████████| 800/800 [16:03<00:00,  1.20s/it]


In [30]:
pd.DataFrame({"answer": predictions})["answer"].apply(pd.Series).to_csv("/content/dev_answers.csv", index=False)

In [31]:
with open("/content/dev_answers.txt", "w") as f:
    f.writelines(list(map(lambda x: x.replace("\n", " ") + "\n", predictions)))

In [32]:
datasets = load_dataset("csv", data_files={"test": "test.tsv"}, sep="\t")
test_dataset = datasets["test"]

Using custom data configuration default-6a7f9824ce1b9f31


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-6a7f9824ce1b9f31/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
beam_count = 5

predictions = []
    
for i in tqdm(test_dataset["toxic_comment"]):
    options = ppln(
        {"toxic_comment": i},
        do_sample=False,
        num_beams=beam_count,
        num_return_sequences=beam_count,
    )

    options = [i["generated_text"].replace("<pad>", "") for i in options]
    answer = sorted(options, key=len)[-1]  # get longest answer
    predictions.append(answer)

  0%|          | 0/875 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:978: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
100%|██████████| 875/875 [17:24<00:00,  1.19s/it]


In [34]:
pd.DataFrame({"answer": predictions})["answer"].apply(pd.Series).to_csv("/content/test_answers.csv", index=False)

In [35]:
with open("/content/test_answers.txt", "w") as f:
    f.writelines(list(map(lambda x: x.replace("\n", " ") + "\n", predictions)))